**Create an OpenStudio Analysis JSON (OSA)**

The following is taken from: https://www.tandfonline.com/doi/full/10.1080/19401493.2020.1778788

The OpenStudio Analysis Framework (OSAF) was developed with the intention of putting advanced energy modelling capabilities in the hands of industry practitioners. To do this, OSAF uses analysis workflows that are shareable, transportable, and reproducible by using a standardized OSA problem definition JSON file, and supporting files that can be easily compressed and used on different computing environments. The OSA contains information on the algorithm and its parameters, variables with their uncertainty descriptions (e.g. the minimum and maximum allowed values, normally or uniformly distributed), output variables, and objective functions.

In these workflows, individual models are no longer the basis from which the analysis is derived. Instead, the paradigm involves a ‘seed’, which is the baseline model around which the analysis is built, and a chain of scripts, which are run by the Workflow Gem and are used to programmatically implement changes to the baseline model. The scripts, called Measures (Roth, Goldwasser, and Parker 2016; Brackney et al. 2018; Roth et al. 2018), can be generic and reusable across different models and analyses. The chain of scripts and their arguments, as well as the definition of the variables, is described in the ‘workflow’ section of the OSA.

The Measure-based workflow can be described abstractly as a black box mathematical function  

    y=F(x)        equation (1)    

where x is a vector of independent variables or inputs, y is a vector of output variables, and the function F is the application of Measures on the seed model followed by the simulation of the building energy model and any reporting Measures that are part of the workflow and used to compute the output variables. Formulating the problem in this way enables the application and switching between the algorithms that operate on the entire BEM based workflow, since the algorithms are designed to operate on processes which can be defined by equation (1). In this context, an independent variable x is simply a Measure input argument that is no longer fixed to a specific value, but one that has an allowed range of values and an uncertainty distribution type. To compute the outputs y, the selected algorithm creates a set of specific x variable values (based on the problem type and variable uncertainty descriptions) along with the information in the workflow section, and eventually gets transformed to a set of specific OSWs, which are then simulated.

![Image description](OSA.png)

The baseline model changes can be varied manually by the user or automatically by an algorithm. Although manually configured analyses are limited by nature in the number of design alternatives they contain, algorithmic analyses can have a much larger number of design alternatives. The use of the same simulation workflow is key to enabling different algorithmic types of analyses (i.e. change from full factorial to sensitivity analysis to uncertainty quantification to optimization to calibration).

This example will create an OSA that can generate a similiar Pareto Front graphic like the following:

![Image description](heating_cooling_pareto.png)

OSAs can be created using the GUI OpenStudio Parametric Analysis Tool (PAT) or programatically using the OpenStudio-Analysis gem, which is the focus of this notebook.
To start, load the openstudio-analysis gem.  The features described here require Version >= 1.3.0-0

In [1]:
#gem 'openstudio-analysis', '1.3.3'
require 'openstudio-analysis'

OpenStudio::Analysis::VERSION

Top level ::CompositeIO is deprecated, require 'multipart/post' and use `Multipart::Post::CompositeReadIO` instead!
Top level ::Parts is deprecated, require 'multipart/post' and use `Multipart::Post::Parts` instead!


"1.3.0"

We previously defined our workflow in the OSW 'optimization_workflow.osw'. We can use that as a starting point and turn that workflow into an OSA.
To start, create an OpenStudio::Analysis object and call it 'Optimization Example'.
Next, call the .convert_osw function on the OSW to move the [:steps] section of the OSW into the [:problem][:workflow] section of the OSA.

In [2]:
analysis = OpenStudio::Analysis.create('Optimization Example')
analysis.convert_osw('optimization_workflow.osw')

# Or use an array of search paths to find measures

#paths = [
#        'spec/files/measures',
#        'spec/files/measures_second_path'
#        ]
#analysis.convert_osw('calibration_workflow.osw', paths)

[{:arguments=>{:r_value_perc_change=>10.0}, :measure_dir_name=>"measures/ExteriorWallThermalPropertiesPercentChange"}, {:arguments=>{:r_value_perc_change=>10.0}, :measure_dir_name=>"measures/RoofThermalPropertiesPercentChange"}, {:arguments=>{:coil=>"*All Gas Heating Coils*", :coil_efficiency_perc_change=>10.0}, :measure_dir_name=>"measures/CoilHeatingGasPercentChange"}, {:arguments=>{:coil=>"*All DX Cooling Coils*", :rated_cop_perc_change=>10.0}, :measure_dir_name=>"measures/CoilCoolingDXSingleSpeedPercentChange"}, {:arguments=>{:fan=>"*All Fans*", :fan_efficiency_perc_change=>10.0}, :measure_dir_name=>"measures/FansPercentChange"}, {:arguments=>{:ElectricEquipment_perc_change=>10.0, :lights_perc_change=>10.0, :space=>"*All Spaces*", :space_type=>"*All SpaceTypes*"}, :measure_dir_name=>"measures/GeneralCalibrationMeasurePercentChange"}, {:arguments=>{}, :measure_dir_name=>"measures/openstudio_results"}]

Next we will define Output Variables and Objective Functions.  Notice that the first four Output Variables below are turned into Objective Functions by setting the :objective_function key to `True`.  Multi-Objective algorithms have the ability to group Objective Functions together in the sense that they will be added together and treated as a single value, typically using the L2 Norm.  The default value for key :objective_function_group is 1, so to make them separate groups so that they can be optimized or treated separately we will change their values to 2,3,4.  

In [3]:
o = analysis.add_output(
      display_name: 'electricity_cooling_ip',
      display_name_short: 'electricity_cooling_ip',
      name: 'openstudio_results.electricity_cooling_ip',
      units: 'kWh',
      objective_function: true
    )

o = analysis.add_output(
      display_name: 'natural_gas_heating_ip',
      display_name_short: 'natural_gas_heating_ip',
      name: 'openstudio_results.natural_gas_heating_ip',
      units: 'MBtu',
      objective_function: true,
      objective_function_group: 2
    )

o = analysis.add_output(
      display_name: 'annual_peak_electric_demand',
      display_name_short: 'annual_peak_electric_demand',
      name: 'openstudio_results.annual_peak_electric_demand',
      units: 'kW',
      objective_function: true,
      objective_function_group: 3
    )

o = analysis.add_output(
      display_name: 'electricity_ip',
      display_name_short: 'elec_ip',
      name: 'openstudio_results.electricity_ip',
      units: 'kWh',
      objective_function: true,
      objective_function_group: 4
    )

[{:units=>"kWh", :objective_function=>true, :objective_function_index=>0, :objective_function_target=>nil, :objective_function_group=>1, :scaling_factor=>nil, :visualize=>true, :metadata_id=>nil, :export=>true, :display_name=>"electricity_cooling_ip", :display_name_short=>"electricity_cooling_ip", :name=>"openstudio_results.electricity_cooling_ip"}, {:units=>"MBtu", :objective_function=>true, :objective_function_index=>1, :objective_function_target=>nil, :objective_function_group=>2, :scaling_factor=>nil, :visualize=>true, :metadata_id=>nil, :export=>true, :display_name=>"natural_gas_heating_ip", :display_name_short=>"natural_gas_heating_ip", :name=>"openstudio_results.natural_gas_heating_ip"}, {:units=>"kW", :objective_function=>true, :objective_function_index=>2, :objective_function_target=>nil, :objective_function_group=>3, :scaling_factor=>nil, :visualize=>true, :metadata_id=>nil, :export=>true, :display_name=>"annual_peak_electric_demand", :display_name_short=>"annual_peak_electri

A static Measure Argument can be turned into a Variable by assigning a likelihood distribution type to it along with parameters that best describe its variability.  As an example, we could describe a variable with equal probability of taking values from 0 to 10 as a `uniform` distribution with a `:minimum` value of `0` and a `:maximum` value of `10`.  

The distribution is described by a hash with the following keys, not all of them are required:  

`:type` of distributions are: `discrete`, `uniform`, `triangle`, `normal`, `lognormal`, `integer_sequence`. **required**  
`:minimum` Minimum value of the distribution, **required** for all distributions  
`:maximum` Maximum value of the distribution, **required** for all distributions  
`:standard_deviation` The standard deviation, if the distribution requires it.  
`:mode` The mean/mode of the distribution (if required)  
`:mean` Alias for the mode. If this is used it will override the mode  
`:relation_to_output` How is the variable correlates to the output of interest (for continuous distributions)  
`:step_size` Minimum step size (delta_x) of the variable (for continuous distributions)  
`:values` If discrete, then the values to run  
`:weights` If discrete, then the weights for each of the discrete values, must be the same length as values, and sum to 1. If empty, then it will create this automatically to be uniform.  

The steps to making a Variable is to  
  1. find the Measure in the workflow.  
  2. create distribution hash.  
  3. call `.make_variable` with the name of the argument that will become a variable and the name you wish to call the variable, along with the distribution hash. 


In [4]:
m = analysis.workflow.find_measure('exterior_wall_thermal_properties_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('r_value_perc_change', 'Exterior wall total R-value Percent Change', d)

m = analysis.workflow.find_measure('roof_thermal_properties_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('r_value_perc_change', 'Roof total R-value Percent Change', d)

m = analysis.workflow.find_measure('coil_heating_gas_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('coil_efficiency_perc_change', 'Percent Change for heating coil Efficiency', d)

m = analysis.workflow.find_measure('coil_cooling_dx_single_speed_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('rated_cop_perc_change', 'Percent Change for DX COP', d)

m = analysis.workflow.find_measure('fans_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('fan_efficiency_perc_change', 'Percent Change for Fan Efficiency', d)

m = analysis.workflow.find_measure('general_calibration_measure_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('lights_perc_change', 'Lights Percent Change', d)

m = analysis.workflow.find_measure('general_calibration_measure_percent_change')
d = {
      type: 'uniform',
      minimum: -50,
      maximum: 50,
      mean: 0
    }
    m.make_variable('ElectricEquipment_perc_change', 'Electric Equipment Percent Change', d)

true

Server scripts are bash scripts that can be used for various purposes such as changing the Gems that are used in an analysis or sending results to an S3 bucket.  There are 2 types of scripts: `Analysis` and `DataPoint` and they can be either `initialization` or `finalization`.  As an example, we have a dummy script that simply prints a one, two, three or four and various locations in the analysis.  These scripts need to be added so that the scripts are added to the .zip payload that gets submitted to the Server along with the Measures, seed models, etc. The path to the scripts is the first argument, an array of arguments for the script is next, followed by when the script should run.

In [5]:
#full_path_to_file = File.expand_path("scripts/script.sh", path_to_this_file)
analysis.server_scripts.add('scripts/script.sh', ['one', 'two'])
# add analysis finalization script
analysis.server_scripts.add('scripts/script.sh', ['three', 'four'], 'finalization', 'analysis')  

true

In [6]:
#default analysis is single_run, change to nsga_nrel
analysis.analysis_type = 'nsga_nrel'
#number_of_samples
analysis.algorithm.set_attribute('number_of_samples', 20)
#generations
analysis.algorithm.set_attribute('generations', 10)
#tournament_size
analysis.algorithm.set_attribute('tournament_size', 5)
#max_queued_jobs
analysis.algorithm.set_attribute('max_queued_jobs', 20)
File.write('analysis.json',JSON.pretty_generate(analysis.to_hash))

40451

In [7]:
analysis.save_osa_zip('analysis.zip', all_weather_files = false, all_seed_files = false)

osw_path: C:/Projects/Notebooks/osw_project/optimization_workflow.osw
osw_full_path: C:/Projects/Notebooks/osw_project
Adding Support Files: Weather
  Adding weather/USA_CO_Golden-NREL.724666_TMY3.epw
Adding Support Files: Seed Models
  Adding C:/Projects/Notebooks/osw_project/Optimization/seeds/example_model.osm
Adding Support Files: Libraries
Adding Support Files: Server Scripts
  Adding C:/Projects/Notebooks/osw_project/scripts/script.sh as scripts/data_point/initialization.sh
  Adding arguments as scripts/data_point/initialization.args
  Adding C:/Projects/Notebooks/osw_project/scripts/script.sh as scripts/analysis/finalization.sh
  Adding arguments as scripts/analysis/finalization.args
Adding Measures
  Adding ExteriorWallThermalPropertiesPercentChange
    Adding File C:/Projects/Notebooks/osw_project/measures/ExteriorWallThermalPropertiesPercentChange/LICENSE.md
    Adding File C:/Projects/Notebooks/osw_project/measures/ExteriorWallThermalPropertiesPercentChange/measure.rb
    Ad

[#<OpenStudio::Analysis::WorkflowStep:0x000000000c956620 @name="exterior_wall_thermal_properties_percent_change", @display_name="Exterior Wall Thermal Percent Change", @type="ModelMeasure", @measure_definition_class_name="ExteriorWallThermalPropertiesPercentChange", @measure_definition_directory="./measures/ExteriorWallThermalPropertiesPercentChange", @measure_definition_directory_local="C:/Projects/Notebooks/osw_project/measures/ExteriorWallThermalPropertiesPercentChange", @measure_definition_display_name="Exterior Wall Thermal Percent Change", @measure_definition_name="exterior_wall_thermal_properties_percent_change", @measure_definition_name_xml=nil, @measure_definition_uuid="6ab39cf2-1f9e-4570-a797-360a814aaa64", @measure_definition_version_uuid="e6162515-fa62-411a-9f4f-ef27fec69598", @uuid="6ab39cf2-1f9e-4570-a797-360a814aaa64", @version_uuid="e6162515-fa62-411a-9f4f-ef27fec69598", @description="Change exterior walls by altering the thermal resistance, density, and solar absorptan